# Tour API로 제주도에 있는 관광지 정보 추출

In [1]:
import pandas as pd
import requests
import json
import os

In [3]:
csvs = os.listdir('data/제주_관광_데이터/제주 관광수요 예측 데이터_비짓제주 요일별 데이터')

In [5]:
csvs.remove("제주 관광수요예측 데이터_비짓제주_요일별_컬럼정의서.xls")

In [16]:
df1 = pd.read_csv('data/제주_관광_데이터/제주 관광수요 예측 데이터_비짓제주 요일별 데이터/JT_WKDAY_ACCTO_TRRSRT_SCCNT_LIST_202308.csv')
df2 = pd.read_csv('data/JEJU_MCT_DATA.csv',encoding='cp949')

In [11]:
df1['AREA_NM']

0        브랭섬홀아시아 아이스링크
1       (주)제주해양레저 투명카약
2          1100고지(휴게소)
3             1100고지습지
4               1112도로
             ...      
4525               흑소랑
4526             흑염소마루
4527            흑한우명품관
4528              희신이네
4529                힌즈
Name: AREA_NM, Length: 4530, dtype: object

In [2]:
api_key = ## '6Oa%2FW5VE%2FlS9M1L1C9Lhp%2FVWp5ugIonT7A%2BdGNaG0lzy88CLt5ALE%2Ff8awzc15fZREcqpcxJaLrFtF%2FnTrPjug%3D%3D'

In [ ]:
# 음식 39 쇼핑 38 문화시설 14 관광지 12 숙박 32
# jeju 39

In [15]:
#content_type=32
url = f"http://apis.data.go.kr/B551011/KorService1/areaBasedList1?serviceKey={api_key}&listYN=N&areaCode=39&MobileOS=ETC&MobileApp=AppTest&_type=json"
response = requests.get(url)
print(response.text)

{"response": {"header":{"resultCode":"0000","resultMsg":"OK"},"body": {"items": {"item":[{"totalCnt":2414}]},"numOfRows":1,"pageNo":1,"totalCount":1}}}


In [4]:
contents_list=[]

for page_num in range(1, (2414//50)):
    #content_type = '32'

    numOfRows = '50'
    url = f"http://apis.data.go.kr/B551011/KorService1/areaBasedList1?serviceKey={api_key}&numOfRows={numOfRows}&pageNo={page_num}&areaCode=39&MobileOS=ETC&MobileApp=AppTest&_type=json"
    response = requests.get(url)
    contents = response.text
    contents = json.loads(contents)
    contents_list.append(contents)


In [24]:
#!pip install openpyxl
df = pd.read_excel('type.xlsx')

In [25]:
df = df.iloc[3:,1:]
columns = df.iloc[0,:].values
rename_dict = {}
for i,j in zip(df.columns, columns):
    rename_dict[i] = j

df.rename(columns=rename_dict, inplace=True)
df = df.iloc[1:,:]
df

,대분류 (cat1),중분류 (cat2),소분류 (cat3),대분류,중분류,소분류
4,A01,A0101,A01010100,자연,자연관광지,국립공원
5,A01,A0101,A01010200,자연,자연관광지,도립공원
6,A01,A0101,A01010300,자연,자연관광지,군립공원
7,A01,A0101,A01010400,자연,자연관광지,산
8,A01,A0101,A01010500,자연,자연관광지,자연생태관광지
...,...,...,...,...,...,...
151,A05,A0502,A05020300,음식,음식점,일식
152,A05,A0502,A05020400,음식,음식점,중식
153,A05,A0502,A05020700,음식,음식점,이색음식점
154,A05,A0502,A05020900,음식,음식점,카페/전통찻집


In [40]:
content_dict = {"12":"관광지",
                "14":"문화시설",
                "32":"숙박",
                "38":"쇼핑",
                "39":"음식점",
                "15":"행사/공연/축제",
                "25":"여행코스",
                "28":"레포츠",
                }

In [26]:
cat1_dict={}
cat2_dict={}
cat3_dict={}
for values in df.values:
    cat1_dict[values[0]] = values[3]
    cat2_dict[values[1]] = values[4]
    cat3_dict[values[2]] = values[5]

In [54]:
import re

def find_text_between_special_characters(text):
    pattern = r'(\(|\[)([^\(\)\[\]]+?)(\)|\])'
    matches = re.findall(pattern, text)
    return [''.join(match) for match in matches]

contents_dict={}
for content in contents_list:
    if content['response']['body']['items']:
        x = content['response']['body']['items']['item']

        for i in x:
            detail_dict = {}
            #detail_dict['title'] = i['title']
            detail_dict['addr'] = i['addr1'] +" " +i['addr2']
            detail_dict['contenttypeid'] = content_dict[i['contenttypeid']]
            detail_dict['cat1'] = cat1_dict[i['cat1']]
            detail_dict['cat2'] = cat2_dict[i['cat2']]
            detail_dict['cat3'] = cat3_dict[i['cat3']]
            detail_dict['booktour'] = i['booktour']

            cut = find_text_between_special_characters(i['title'])
            if cut:
                title = i['title'].replace(cut[0],'')
            else:
                title = i['title']
            title = title.replace(' ','')
            
            contents_dict[title] = detail_dict


In [55]:
meta_data = {'numOfContents':len(contents_dict), 'contents':contents_dict}

In [56]:
json_name = 'jeju_total.json'

In [57]:
with open(json_name, 'w') as json_file:
    json.dump(meta_data, json_file, ensure_ascii=False, indent='\t')